In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df1=pd.read_csv('/home/satyam/Downloads/Bengaluru_House_Data.csv')
df1.head()

In [ ]:
df1.dtypes

In [ ]:
df1.shape

In [ ]:
 df1.groupby('area_type')['area_type'].agg('count')

In [ ]:
df2=df1.drop(['area_type','society','balcony','availability'],axis='columns')
df2.head()

In [ ]:
df2.isnull().sum()

In [ ]:
#Removing null values 
#since our data set is very large so we can simply remove these small null values 
df3=df2.dropna()
df3.isnull().sum()

In [ ]:
df3.shape

## Data Cleaning

In [ ]:
 df3['size'].unique()

In [ ]:
 df3['bhk']=df3['size'].apply(lambda x:int(x.split(' ')[0]))

In [ ]:
df3.head()

In [ ]:
df3.drop(['size'],axis=1,inplace=True)

In [ ]:
df3.head()

In [ ]:
df3['bhk'].unique()

In [ ]:
df3[df3['bhk']>20]

In [ ]:
df3.total_sqft.unique()

In [ ]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [ ]:
df3[~df3['total_sqft'].apply(is_float)].head(10)

In [ ]:
def convert_sqft_to_num(x):
    tokens=x.split('-')
    if len(tokens)==2:
        return (float(tokens[0])+(float(tokens[1])))/2
    try:
        return float(x)
    except:
        return None

In [ ]:
df4=df3.copy()
df4['total_sqft']=df4['total_sqft'].apply(convert_sqft_to_num)

In [ ]:
df4.loc[30]

## Feature Engineering and Dimension Reduction

In [ ]:
#creating a new column
df5=df4.copy()
df5['price_per_sqft']=df5['price']*100000/df5['total_sqft']
df5.head()

In [ ]:
len(df5.location.unique())

In [ ]:
#To remove the white spaces from begining and end 
df5.location=df5.location.apply(lambda x:x.strip())


In [ ]:
len(df5.location.unique())

In [ ]:
location_stats=df5.groupby('location')['location'].agg('count').sort_values(ascending=False)
location_stats

In [2]:
len(location_stats[location_stats<=10])

NameError: name 'location_stats' is not defined

In [ ]:
location_stats_less_than_10=location_stats[location_stats<=10]
location_stats_less_than_10

In [ ]:
#Considering all the location whose aggregate count is less than equal to 10 as an other location section
df5.location=df5.location.apply(lambda x:'other' if x in location_stats_less_than_10 else x)
df5[df5.location=='other']

In [ ]:
len(df5.location.unique())

In [ ]:
df5.head(10)

In [ ]:
(df5.location=='other').agg('count')

## Outlier Detection and Removal

In [ ]:
#Assuming a threshold of total_sqft per bedroom as 300
df5[df5.total_sqft/df5.bhk<300].head()

In [ ]:
df5.shape

In [ ]:
df6=df5[~(df5.total_sqft/df5.bhk<300)]
df6.head()

In [ ]:
df6.shape

In [ ]:
df6.price_per_sqft.describe()

In [ ]:
df6.groupby('location').head()

In [ ]:
#If the dataset is in normal distribution then 68% of points in the dataset lies between mean and 1 standard deviation, so filtering anything beyond 1 Standard deviation
def remove_pps_outliers(df):
    df_out=pd.DataFrame()
    for key,subdf in df.groupby('location'):
        #print(subdf)
        #print(key)
        m=np.mean(subdf.price_per_sqft)
        st=np.std(subdf.price_per_sqft)
        reduced_df=subdf[(subdf.price_per_sqft>(m-st))&(subdf.price_per_sqft<=(m+st))]
        df_out=pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out
   

In [ ]:
df7=remove_pps_outliers(df6)
df7.shape

In [ ]:
df7.groupby('bhk').head(10)

In [ ]:
import matplotlib 
matplotlib.rcParams["figure.figsize"] = (15,10)


In [ ]:
#checking if for same total_sqft the price of 2bhk is higher than 3bhk ,so these must be some outliers so performing data cleaning
def plot_scatter_chart(df,location):
    bhk2=df[(df.location==location) & (df.bhk==2)]
    bhk3=df[(df.location==location) & (df.bhk==3)]
    matplotlib.rcParams['figure.figsize']=(15,10)
    plt.scatter(bhk2.total_sqft,bhk2.price,color='blue',label='2 BHK',s=50)
    plt.scatter(bhk3.total_sqft,bhk3.price,color='green',label='3 BHK',s=50,marker='+')
    plt.xlabel('Total_sqft Area')
    plt.ylabel('Price')
    plt.legend()

plot_scatter_chart(df7,'Hebbal')


#### Now we can remove those 2 BHK apartments whose price_per_sqft is less than mean price_per_sqft of 1 BHK apartment

In [ ]:


def remove_bhk_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sqft),
                'std': np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')
df8 = remove_bhk_outliers(df7)






In [ ]:
df8.shape

#### Plot same scatter chart again to visualize price_per_sqft for 2 BHK and 3 BHK properties


In [ ]:
def plot_scatter_chart(df,location):
    bhk2=df[(df.location==location) & (df.bhk==2)]
    bhk3=df[(df.location==location) & (df.bhk==3)]
    matplotlib.rcParams['figure.figsize']=(15,10)
    plt.scatter(bhk2.total_sqft,bhk2.price,color='blue',label='2 BHK',s=50)
    plt.scatter(bhk3.total_sqft,bhk3.price,color='green',label='3 BHK',s=50,marker='+')
    plt.xlabel('Total_sqft Area')
    plt.ylabel('Price')
    plt.legend()

plot_scatter_chart(df8,'Hebbal')

### Now checking that how many data points lie between count and price per sqft 


In [ ]:
matplotlib.rcParams['figure.figsize']=(20,10)
plt.hist(df8.price_per_sqft,rwidth=0.8)
plt.xlabel('price per square feet')
plt.ylabel('count')

## Removing outliers from bathroom feature

In [3]:
df8.bath.unique()

NameError: name 'df8' is not defined

In [ ]:
plt.hist(df8.bath,rwidth=0.8)
plt.xlabel('number of bathrooms')
plt.ylabel('count')

### Checking for outliers

In [ ]:
df8[df8.bath>10]

### Checking those having 2 more bathrooms than the number of bedrooms

In [ ]:
df8[df8.bath>df8.bhk+2]

In [ ]:
#df9=df8[~(df8.bath>df8.bhk+2)]
df9 = df8[df8.bath<df8.bhk+2]
df9.shape

In [ ]:
df9.head(2)

In [ ]:
df10=df9.drop(['price_per_sqft'],axis=1)
df10.head()

## For ML model we need to convert all the string values to the numerical values using ONE HOT ENCODING METHOD OR GET DUMMIES METHOD

In [ ]:
dummies=pd.get_dummies(df10.location)
dummies.head(3)

In [ ]:
df11=pd.concat([df10,dummies.drop('other',axis=1)],axis=1)
df11.head()

In [ ]:
df12=df11.drop('location',axis=1)
df12.head()

## Building a Model 

In [ ]:
df12.shape

In [ ]:
X=df12.drop('price',axis=1)
X.head()

In [ ]:
X.shape

In [ ]:
y=df12['price']

In [ ]:
y.head()

In [ ]:
len(y)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train,y_train)
lr.score(X_test,y_test)

## Using K Fold cross validation to measure accuracy of our Linear Regression Model

## Using K Fold cross validation to measure accuracy of our Linear Regression Model

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv= ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
cross_val_score(LinearRegression(),X,y,cv=cv)

## For hyperparameter tunning and finding best model depending upon the model score we use GridSearch CV

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

def find_best_model_using_gridsearchcv(X,y):
    algos={'linear_regression':{
        'model':LinearRegression(),'params':{'normalize':[True,False]
            
        }
    },
        'lasso':{
            'model':Lasso(),'params':{
                'alpha':[1,2],
                'selection':['random','cyclic']
            }
        },
           'decision_tree':{
               'model':DecisionTreeRegressor(),
               'params':{
                   'criterion':['mse','friedman_mse'],
                   'splitter':['best','random']
               }
           }
        
    }
## Using Cross Validation 
    scores=[]
    ## in order to get random samples from the data set we shuffle the data set using ShuffleSplit
    cv=ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
    for algo_name,config in algos.items():
        gs=GridSearchCV(config['model'],config['params'],cv=cv,return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model':algo_name,
            'best_score':gs.best_score_,
            'best_params':gs.best_params_
        })
    
    return pd.DataFrame(scores,columns=['model','best_score','best_params'])
find_best_model_using_gridsearchcv(X,y)

    

#### From above results we can say that Linear Regression model is the best one for us to use since it shows the best score in comparision to others

## Testing the model

In [4]:
def predict_price(location,sqft,bath,bhk):
    loc_index=np.where(X.columns==location)[0][0]
    print('loc_index is {}'.format(loc_index))
    x=np.zeros(len(X.columns))
    x[0]=sqft
    x[1]=bath
    x[2]=bhk
    if loc_index>=0:
        x[loc_index]=1
    return lr.predict([x])[0]

In [5]:
predict_price('Indira Nagar',1000,2,2)

NameError: name 'np' is not defined

In [ ]:
predict_price('Indira Nagar',1000, 3, 3)



In [ ]:
predict_price('1st Phase JP Nagar',1000, 3, 3)

## Exporting the model to a pickle file

In [ ]:
import pickle
with open('banglore_home_prices_model.pickle','wb') as f:
    pickle.dump(lr,f)

## Exporting location and column information to a json file

In [ ]:
import json
columns={
    'data_columns':[col.lower() for col in X.columns]
}
with open('columns.json','w') as f:
    f.write(json.dumps(columns))